In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

import requests as re
from html.parser import HTMLParser

sns.set_context('poster')
sns.set_style('whitegrid')

In [5]:
df = pd.read_json('./rawData/train_data.json')

In [6]:
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('-', '_') for c in df.columns]
df.columns = [c.replace('/', '_') for c in df.columns]

In [7]:
df['bedrooms'] = df['bedrooms'].apply(float)
df['bedrooms'].fillna(0, inplace=True)
df['bathrooms'].fillna(0, inplace=True)

In [8]:
df.drop(['index', 'level_0'], axis=1, inplace=True)
df['interestVal'] = df['interest_level'].map({'high': 1, 'medium': 0.5, 'low':0})

In [9]:
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

In [10]:
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [11]:
def descrClean(x):
    des = strip_tags(x)
    return des.lower()

In [12]:
df['description'] = df['description'].apply(descrClean)

In [13]:
# Verify two laundry in buildings are the same
df['Laundry_In_Building'].value_counts()

0    39710
1     2208
Name: Laundry_In_Building, dtype: int64

In [14]:
# Verify two laundry in buildings are the same
df['Laundry_in_Building'].value_counts()

0    28025
1    13893
Name: Laundry_in_Building, dtype: int64

In [15]:
def combine_laundry(col1, col2):
    if col1 != 0:
        return col1
    else:
        return col2

In [16]:
# Aggregate to create one laundry in building column that isn't case sensitive
df['laundry_in_building'] = df.apply(lambda row: combine_laundry(row['Laundry_in_Building'], row['Laundry_In_Building']), axis=1)

# Drop old laundry in building columns
df = df.drop(['Laundry_in_Building', 'Laundry_In_Building'], axis=1)

In [17]:
# Drop photos column (is just links to photos)
df = df.drop(['photos'], axis=1)

In [18]:
df.to_json('./rawData/cleaned.json')